# Table of content

1.Download and Explore Dataset

2.Explore Neighborhoods in Toronto city

3.Analyze Each Neighborhood

4.Cluster Neighborhoods

5.Examine Cluster Neighbourhood

## Introduction
Introduction
Exploring Toronto Neighborhoods to open a Restorant As a part of the IBM Data Science professional program Capstone Project, we worked on the real datasets to get an experience of what a data scientist goes through in real life. Main objectives of this project were to define a business problem, look for data in the web and, use Foursquare location data to compare different neighborhoods of Toronto to figure out which neighborhood is suitable for starting a new restaurant business. In this project, we will go through all the process in a step by step manner from problem designing, data preparation to final analysis and finally will provide a conclusion that can be leveraged by the business stakeholders to make their decisions.


 ## Description of the Business Problem & Discussion of the Background (Introduction Section):
Problem Statement:Prospects of opening an Ethiopian Restaurant in Toronto, Canada.
Toronto, the capital of the province of Ontario, is the most populous Canadian city. the city has 103 neighbourhoods and 10 boroughs.so,In this project we will go through step by step process to make a decision whether it is a good idea to open an Indian restaurant. We analyze the neighborhoods in Toronto to identify the most profitable area since the success of the restaurant depends on the people and ambience. Since we already know that Toronto shelter a greater number of ethiopians than any other city in Canada, it is a good idea to start the restaurant here, but we just need to make sure whether it is a profitable idea or not. If so, where we can place it, so it yields more profit to the owner.Description of the Business Problem & Discussion of the Background (Introduction Section):

### Target Audience
This study targets Business personnel who wants to invest or start a restorant work in toronto and the vicinities. It might  also be useful for Data scientists who wish to analyze the neighborhoods of Toronto using Exploratory Data Analysis and other statistical & machine learning techniques to obtain all the necessary data, perform some operations on it and, finally be able to tell a story out of it.



### Data acquisition and cleaning:
Data Sources
a) I’m using “List of Postal code of Canada: M” (https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M) wiki page to get all the informaThis study targets Business personnel who wants to invest or start a restorant work in toronto and the vicinities. it may useful for Data scientists who wish to analyze the neighborhoods of Toronto using Exploratory Data Analysis and other statistical & machine learning techniques to obtain all the necessary data, perform some operations on it and, finally be able to tell a story out of it.

tion about the neighborhoods present in Toronto. This page has the postal code, borough & the name of all the neighborhoods present in Toronto.
b) Then I’m using “https://cocl.us/Geospatial_data” csv file to get all the geographical coordinates of the neighborhoods.

d) To get location and other information about various venues in Toronto

### Exploring and cluster the neighborhoods in Toronto. we begin by importing all the neccesaary libraries
#### Download and Explore Dataset


In [228]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm

import matplotlib.colors as colors
#import k-means from clustering stage
#from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
#import folium # map rendering library)

In [229]:
df=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0)[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


# lets ignore the Borough which is Not assigned

#ignore cell with Borough Notassigned 
indexName = df[ df['Borough'] == 'Not assigned' ].index
df.drop(indexName , inplace= True)
df.head()

In [230]:
#Once we left the unassigned borough then we will reindexed the rows

df = df.reset_index()
del df['index']
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


# if neighborhood not assigned , then the neighborhood will be the same as the borough.

df.loc[df['Neighbourhood'] == 'Not assigned','Neighbourhood']= df['Borough']
df.head()


### Concatenate neighborhood with the same postal code in one row separated with a comma & avoide duplicate if any 


In [231]:
result=df.groupby(['Postal Code'], sort= False).agg(','.join)
result.head()

,Borough,Neighbourhood
Postal Code,,
M1A,Not assigned,Not assigned
M2A,Not assigned,Not assigned
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Adding geographical coordinates to the neighborhoods
Next important step is adding the geographical coordinates to these neighborhoods. To do so I’m extracting the data present in the Geospatial Data csv file and I’m combining it with the existing neighborhood dataframe by merging them both based on the postal code.

In [232]:
#loading the geospatial data
df_Toronto=pd.read_csv("http://cocl.us/Geospatial_data")
df_Toronto.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


I’m renaming the columns to match the existing dataframe formed from ‘List of Postal code of Canada: M’ wiki page.After that I’m merging both the dataframe into one by merging on the postal code.

##### if neighborhood not assigned , then the neighborhood will be the same as the borough.

In [233]:
df_Toronto=pd.merge(df,df_Toronto,on ='Postal Code')
df_Toronto= df_Toronto.rename(columns={'Neighbourhood': 'Neighbourhood'})
df_Toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [234]:
df.loc[df['Neighbourhood'] == 'Not assigned','Neighbourhood']= df['Borough']
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [235]:
print(df_Toronto.shape)

(103, 5)


In [ ]:
print('The dataframe has {} boroughs and {} neighbourhoods.'.format(
        len(df_Toronto['Borough'].unique()),
        df_Toronto.shape[0]
    )
)

# 4. Methodology
Data Collection Method: Data collection has been completed using a methodology based on the BeautifulSoup library to extract data frames from the existing webpage. This is the initial stage to collect the data. This data has been utilized for generating word cloud a special feature of Python.

Geographic Location Data Collection Method: The initial data frame has been used to get longitude and latitude of neighborhoods on the Geopy library. Because of frequent data traffic from our kernel (103 POST and 103 GET), and traffic limitation this call does not make a significant impact. Therefore the location data for postal codes have been collected from the Canada Government website.

Map Exploration Method: The Folium library has been utilized to get the map of Toronto.

Nearby Location Data Extraction Methodology: Folium is a powerful library provides nearby venues such as Yoga Studio, Afghan Restaurant, Airport, Airport Food Court, Airport Gate, Airport Lounge, Airport Service, Airport Terminal, American Restaurant, Antique Shop, Thrift / Vintage Store, Toy / Game Store, 'Trail, Train Station, Vegetarian / Vegan Restaurant, Video Game Store, Vietnamese Restaurant, Wine Bar, Wings Joint, and Women's Store.

K-means clustering- K means cluster is applied to the dataset for the exploration of different venues. Here the algorithm formed 5 clusters with the trends. In the first step we have collected the required data: location and type (category) of every restaurant within 6km from Toronto center . We have also *identified different * (according to Foursquare categorization). We have also presented the top 10 venue categories for each cluster.

We will present a map of all such locations but also create clusters (using k-means clustering) of those locations to identify general zones/neighborhoods/addresses which should be a starting point for final 'street-level' exploration and search for optimal venue location by stakeholders.

# 4. Analysis
Let's perform some basic explanatory data analysis and derive some additional info from our raw data. First, let's count the number of restaurants in every area candidate:

The following graph explores the number of neighborhoods present at a given Borough and neighborhood.

In [ ]:
x=df_Toronto['Borough'].unique()
x

In [ ]:
y=df_Toronto['Borough'].unique()
y

In [ ]:
import matplotlib.pyplot as plt
y.plot(kind='bar',figsize=(8,5))
plt.title('bar graph toronto Borough unique numbers')#add title to the bar 
plt.ylabel('Number of Borough')#add y label
plt.xlabel('Names of Borough')# ad x-label
plt.show

In [ ]:
df_Toronto.plot(kind='bar',figsize=(8,5)) 

plt.title('bar graph for Neighbourhoods present at a given Borough.')#add title to the bar 
plt.ylabel('Neighbourhood')#add y label
plt.xlabel('Borough'),#ad x-label
plt.show

In [238]:
import sys
!{sys.executable} -m pip install geocoder

print('Packages installed.')

Packages installed.


In [239]:
!pip install geopy
import geopy
from  geopy.geocoders import Nominatim
import pandas as pd
locator = Nominatim(user_agent="KapilsGeocoder")
location = locator.geocode("Toronto, Canada")
from geopy.extra.rate_limiter import RateLimiter
# PostalCode  Borough  Neighborhood
df_temp=df_Toronto.copy()
# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
# 2- - create location column
df_temp['Address'] = df_temp['Postal Code'].astype(str) + ',' + ' Toronto'   
df_temp['Location'] = df_temp['Address'].apply(geocode)
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
df_temp['Point'] = df_temp['Location'].apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - split point column into latitude, longitude and altitude columns
df_temp[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df_temp['Point'].tolist(), index=df_temp.index)
df_temp # get the Get the latitude and the longitude coordinates of each neighborhood obtained

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Address,Location,Point,latitude,longitude,altitude
0,M3A,North York,Parkwoods,43.753259,-79.329656,"M3A, Toronto","(Toronto, El Peñón, Loba, Bolívar, Caribe, Col...","(8.8748315, -73.9766442, 0.0)",8.874832,-73.976644,0.0
1,M4A,North York,Victoria Village,43.725882,-79.315572,"M4A, Toronto",None,None,NaN,NaN,NaN
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,"M5A, Toronto",None,None,NaN,NaN,NaN
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,"M6A, Toronto",None,None,NaN,NaN,NaN
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,"M7A, Toronto","(Toronto, El Peñón, Loba, Bolívar, Caribe, Col...","(8.8748315, -73.9766442, 0.0)",8.874832,-73.976644,0.0
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242,"M9A, Toronto",None,None,NaN,NaN,NaN
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,"M1B, Toronto","(Toronto, El Peñón, Loba, Bolívar, Caribe, Col...","(8.8748315, -73.9766442, 0.0)",8.874832,-73.976644,0.0
7,M3B,North York,Don Mills,43.745906,-79.352188,"M3B, Toronto",None,None,NaN,NaN,NaN
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,"M4B, Toronto",None,None,NaN,NaN,NaN
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,"M5B, Toronto",None,None,NaN,NaN,NaN


In [240]:
address = "Toronto, ON"
geolocator = Nominatim(user_agent="toronto_explorer") # Changed user agent due to collision
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto city are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Toronto city are 43.6534817, -79.3839347.


### Create a map of the whole Toronto City with neighborhoods superimposed on top

In [241]:
!pip install folium

import folium # map rendering library
 
# import k-means from clustering stage
from sklearn.cluster import KMeans
 
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
map_toronto

## Add markers to the map.

In [242]:

# create map of New York using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df_Toronto['Latitude'], df_Toronto['Longitude'], df_Toronto['Borough'], df_Toronto['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto


### Foursquare
Now that we have our location candidates, let's use Foursquare API to get info on restaurants in each neighborhood.

We're interested in venues in 'food' category, but only those that are proper restaurants - coffe shops, pizza places, bakeries etc. are not direct competitors so we don't care about those. So we will include in out list only venues that have 'restaurant' in category name, and we'll make sure to detect and include all the subcategories of specific 'Italian restaurant' category, as we need info on Italian restaurants in the neighborhood.



### Define Foursquare Credentials and Version

In [243]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [244]:
CLIENT_ID = 'LV5DTW214RMO4C1KPG5PGERTLJTCA1PIBYTFYPNEQH231DI2' # your Foursquare ID
CLIENT_SECRET = '3YJSV4RELCK2KNFKHJS4ZNS0WTUU2YVB4HIEPLIOTXE15IFQ' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100
radius = 500
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: LV5DTW214RMO4C1KPG5PGERTLJTCA1PIBYTFYPNEQH231DI2
CLIENT_SECRET:3YJSV4RELCK2KNFKHJS4ZNS0WTUU2YVB4HIEPLIOTXE15IFQ


#### Explore the first neighborhood in our data frame "df_toronto_denc"

In [245]:

df_Toronto_denc = df_Toronto[df_Toronto['Borough'].str.contains("Toronto")].reset_index(drop=True)
neighborhood_name = df_Toronto_denc.loc[0, 'Neighbourhood']
print(f"The first neighbourhood's name is '{neighborhood_name}'.")


The first neighbourhood's name is 'Regent Park, Harbourfront'.


#### Get the neighborhood's latitude and longitude values.


In [246]:
neighborhood_latitude = df_Toronto_denc.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_Toronto_denc.loc[0, 'Longitude'] # neighborhood longitude value
 
print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


##### Now, let's get the top 100 venues that are in The Beaches within a radius of 500 meters

In [247]:
radius = 500 # define radius
LIMIT = 100 # limit of number of venues returned by Foursquare API
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ffa167c9661b37fc44f30bb'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 46,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
 

#### Now we are ready to clean the json and structure it into a pandas dataframe

###### Before we proceed, let's borrow the get_category_type function from the Foursquare lab.

###### function that extracts the category of the venue

In [248]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [249]:

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149
4,Body Blitz Spa East,Spa,43.654735,-79.359874


In [250]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

46 venues were returned by Foursquare.


###  Explore neighborhoods in a part of Toronto City
We are working on the data frame df_toronto_denc. Recall that, this region contain DENC of Toronto where,

In [251]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
 
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#####  Now write the code to run the above function on each neighborhood and create a new dataframe called toronto_denc_venues


In [ ]:

Toronto_denc_venues = getNearbyVenues(names=df_Toronto_denc['Neighbourhood'],
                                   latitudes=df_Toronto_denc['Latitude'],
                                   longitudes=df_Toronto_denc['Longitude']
                                  )
Toronto_denc_venues.head()

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park


In [ ]:
print(Toronto_denc_venues.shape)

In [ ]:
#check how many venues were returned for each neighborhood
Toronto_denc_venues.groupby('Neighbourhood').count()

Toronto_denc_venues.shape

In [ ]:
# examine the new dataframe size.

toronto_onehot.shape

In [ ]:
#group rows by neighbourhood and by taking the mean of the frequency of occurrence of each category

toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

In [ ]:
#confirm the new size

toronto_grouped.shape


In [ ]:
# print each neighborhood along with the top 5 most common venues

num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


In [ ]:
# sort the venues in descending order.

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
# create the new dataframe and display the top 10 venues for each neighborhood.

num_top_venues = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

In [ ]:
# Run k-means to cluster the neighborhood into 5 clusters.

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

##### Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [ ]:
Toronto_merged=pd.merge(df_Toronto,neighbourhoods_venues_sorted, on='Neighbourhood')
Toronto_merged.head()

In [ ]:
# Finally, let's visualize the resulting clusters

# create map
map_clusters = folium.Map(location=[43.67635739999999, -79.2930312], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)] 
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array] 

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'],Toronto_merged['Longitude'], Toronto_merged['Neighbourhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

##  Examine Clusters

##### Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.


In [ ]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

In [ ]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

In [ ]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [ ]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

In [ ]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

##  Results and Discussion 
The battel of a neighborhood project explored different categories of venues. In this analysis, initially, a sum of 103 unique postal codes has been observed for 10 diffrent Borough. Using geocoder of Geopy library for Toronto, North York, Parkwoods we get the raw result that are comprehensive and self explanatory in nature.

Based on the above result we split the data into longitude and latitude of the neighborhood. analysis of each neighborhood is carried out with the help of one-hot encoding. The result of one-hot encoding identified 235/236 types of places. Followed by one-hot encoding we explore the top 46 common venues in each of the neighborhoods.

After one hot encoding and venue exploration, we applied the k-means algorithm for k=5. It has identified 5 cluster centroids c0, c1, c2, c3, and c4. These five clusters identified in five different colors. Cluster 0 is having only one neighborhood, cluster 2 is having the maximum number of neighborhoods.



## 7. Conclusion 
The purpose of this project was to identify Toronto areas close to the center with a low number of restaurants to aid stakeholders in narrowing down the search for the optimal location for a new restaurant. By calculating restaurant density distribution from Foursquare data we have first identified general boroughs that justify further analysis (Down Town and East Toronto), and then generated an extensive collection of locations that satisfy some basic requirements regarding existing nearby restaurants. Clustering of those locations was then performed to create major zones of interest (containing the greatest number of potential locations) and addresses of those zone centers were created to be used as starting points for final exploration by stakeholders.

Final decisions on optimal restaurant location will be made by stakeholders based on specific characteristics of neighborhoods and locations in every recommended zone, taking into consideration additional factors like attractiveness of each location (proximity to park or water), levels of noise/proximity to major roads, real estate availability, prices, social and economic dynamics of every neighborhood.